In [1]:
import configparser
import pymongo

config = configparser.ConfigParser()
config.read("credentials.ini")

client = pymongo.MongoClient("mongodb+srv://{}:{}@cluster0-fbceu.mongodb.net/test?retryWrites=true&w=majority".format(config['MONGO']['user'], config['MONGO']['password']))
db = client.hawkerv1

In [5]:
db.dailyListing.update_many({},{"$set":{"zone":"Tembusu"}})

In [6]:
db.transaction.update_many({},{"$set":{"zone":"Tembusu"}})

In [2]:
def fetch_all_hawkers(db, _id=0):
    hawkers = []
    for doc in db.hawker.find({}, {"_id": _id}):
        hawkers.append(doc)
    return hawkers

def fetch_hawker_by_code(db, code, _id=0):
    return db.hawker.find_one({"code": code}, {"_id": _id})

def insert_hawker(db, name, code, address, image):
    if fetch_hawker_by_code(db, code):
        raise Exception("Hawker with code {} already exists!".format(code))
    return db['hawker'].insert_one({
        "name": name,
        "code": code,
        "address": address,
        "image": image
    }).inserted_id

def del_hawker(db, code):
    return db.hawker.delete_one({"code": code}).deleted_count

def update_hawker(db, code, hawker):
    return db.hawker.find_one_and_update({"code": code}, {"$set": hawker})

In [169]:
name = "Telok Blangah Rise Market & Food Centre"
code = "TBR"
address = "36 Telok Blangah Rise, Singapore 090036"
image = ""
#inserted_id = insert_hawker(db, name, code, address, image).inserted_id

#modified = update_hawker(db, "THM", {"address": "48A Tanglin Halt Rd, Singapore 148813"})
x = update_hawker(db, "TBR", {"image": "", "name": "Telok Blangah Rise Market & Food Centre"})

{'_id': ObjectId('5ebf8ea87ae65def16a8c643'),
 'name': 'Telok Blangah Rise Market & Food Centre',
 'code': 'TBR',
 'address': '36 Telok Blangah Rise, Singapore 090036',
 'image': ''}

In [167]:
fetch_hawker_by_code(db, "TBR")

{'name': 'Telok Blangah Rise Market & Food Centre',
 'code': 'TBR',
 'address': '36 Telok Blangah Rise, Singapore 090036',
 'image': ''}

In [3]:
def fetch_stall_by_id(db, stallId, _id=0):
    return db.stall.find_one({"stallId": stallId}, {"_id": _id})

def fetch_stalls_by_location(db, location, _id=0):
    return [doc for doc in db.stall.find({"location": location}, {"_id": _id})]

def insert_stall(db, name, stall_type, location, image, stallNo, food, about, contact):
    stallId = "{}_{}".format(location, stallNo)
    if fetch_stall_by_id(db, stallId):
        raise Exception("Store already exists!")
    stall = {
        "name": name,
        "type": stall_type,
        "location": location,
        "image": image,
        "stallNo": stallNo,
        "stallId": stallId,
        "food": food,
        "about": about,
        "contact": contact
    }
    inserted_stall = db['stall'].insert_one(stall).inserted_id
    return inserted_stall

def del_stall(db, stallId):
    return db.stall.delete_one({"stallId": stallId}).deleted_count

def update_stall(db, stallId, stall):
    return db.stall.find_one_and_update({"stallId": stallId}, {"$set": stall})

In [193]:
name = "Mei Wei Xiao Chi"
stall_type = ["Food", "Dessert"]
location = "THM"
stallNo = "01-02"
food = [
    {
        "name": "Fried Mee Hoon",
        "id": 1,
        "description": "",
        "price": 1.2,
        "image": "",
        "available": True
    },
    {
        "name": "Fried Bee Hoon",
        "id": 2,
        "description": "",
        "price": 1.2,
        "image": "",
        "available": True
    },
    {
        "name": "Fried Mee + Bee Hoon ($1.20)",
        "id": 3,
        "description": "",
        "price": 1.2,
        "image": "",
        "available": True
    },
    {
        "name": "Fried Mee + Bee Hoon ($1.50)",
        "id": 4,
        "description": "",
        "price": 1.5,
        "image": "",
        "available": True
    },
    {
        "name": "Peanut Porridge",
        "id": 5,
        "description": "",
        "price": 1.1,
        "image": "",
        "available": True
    },
    {
        "name": "Red Bean Soup",
        "id": 6,
        "description": "",
        "price": 1.1,
        "image": "",
        "available": True
    },
    {
        "name": "Green Bean Soup",
        "id": 7,
        "description": "",
        "price": 1.1,
        "image": "",
        "available": True
    },
    {
        "name": "Bubur Hitam",
        "id": 8,
        "description": "",
        "price": 1.1,
        "image": "",
        "available": True
    },
    {
        "name": "Chicken Wing",
        "id": 9,
        "description": "",
        "price": 1.1,
        "image": "",
        "available": True
    }
]
about = {
    "recommended": "Bee Hoon, Dessert",
    "description": "",
    "images": []
}
contact = {
    "poc": "",
    "number": "",
    "openingHours": ""
}
stall_id = insert_stall(db, name, stall_type, location, stallNo, food, about, contact)

In [13]:
stalls = fetch_stalls_by_location(db, "THM")

In [20]:
insert_stall(db, stall['name'], stall['type'], stall['location'], stall['image'], stall['stallNo'], stall['food'], stall['about'], stall['contact'])

ObjectId('5ec27af7303aa9d086371271')

In [19]:
stall

{'name': 'Cafe 66',
 'type': ['Drinks'],
 'location': 'THM',
 'stallNo': '01-01',
 'stallId': 'THM_01-01',
 'food': [{'available': True,
   'description': 'Black coffee',
   'id': 1,
   'image': '',
   'name': "Coffee 'O'",
   'price': 0.7},
  {'available': True,
   'description': '',
   'id': 2,
   'image': '',
   'name': 'Coffee w Milk',
   'price': 0.9},
  {'available': True,
   'description': 'Black tea',
   'id': 3,
   'image': '',
   'name': "Tea 'O'",
   'price': 0.7},
  {'available': True,
   'description': '',
   'id': 4,
   'image': '',
   'name': 'Tea w Milk',
   'price': 0.9},
  {'available': True,
   'description': '',
   'id': 5,
   'image': '',
   'name': "Milo 'O'",
   'price': 1},
  {'available': True,
   'description': '',
   'id': 6,
   'image': '',
   'name': 'Milo w Milk',
   'price': 1.1},
  {'available': True,
   'description': '',
   'id': 7,
   'image': '',
   'name': 'Iced Milo',
   'price': 1.7}],
 'about': {'description': 'Coffee stall running since 1966',
 

In [12]:
stall = stalls[2]
stall['about']['urls'] = []
update_stall(db, stall['stallId'], stall)

{'_id': ObjectId('5ec187702cbda5bc4ebfd54b'),
 'name': 'Da Xi Hainanese Chicken Rice Stall',
 'type': ['Food'],
 'location': 'THM',
 'image': '',
 'stallNo': '01-19',
 'stallId': 'THM_01-19',
 'food': [{'id': 1,
   'name': 'Roasted Chicken Rice',
   'price': '3',
   'description': '',
   'image': '',
   'available': True},
  {'id': 2,
   'name': 'Steamed Chicken Rice',
   'price': '3',
   'description': '',
   'image': '',
   'available': True}],
 'about': {'recommended': '', 'description': '', 'images': []},
 'contact': {}}

In [115]:
import datetime

today = datetime.datetime.now().strftime("%d%m%Y")
[doc for doc in db.dailyListing.find({"date": {"$gte": today}}, {"_id": 0})]

[{'name': 'Tanglin Halt Market',
  'code': 'THM',
  'address': '48A Tanglin Halt Rd, Singapore 148813',
  'image': '',
  'date': '16052020',
  'stalls': [{'stallId': 'THM_01-01',
    'name': 'Cafe 66',
    'food': [{'name': "Coffee 'O'",
      'id': 1,
      'description': 'Black coffee',
      'price': 0.7,
      'image': '',
      'available': True,
      'quantity': -1},
     {'name': 'Coffee w Milk',
      'id': 2,
      'description': '',
      'price': 0.9,
      'image': '',
      'available': True,
      'quantity': -1}]}],
  'orderAvailable': True,
  'meal': 'lunch'}]

In [4]:
def fetch_all_listings(db, _id=0):
    today = datetime.datetime.now().strftime("%d%m%Y")
    return [doc for doc in db.dailyListing.find({"date": {"$gte": today}}, {"_id": _id})]

def fetch_listing(db, date, _id=0):
    listing = db.dailyListing.find_one({"date": date}, {"_id": _id})
    listing['stalls'].sort(key=lambda x: x['stallId'])
    listing['stalls'].sort(key=lambda x: x['available'], reverse=True)
    return listing

def insert_listing(db, date, code, meal, orderAvailable=True):
    if fetch_listing(db, date):
        raise Exception("Listing for this date already exists, use update instead!")
    listing = fetch_hawker_by_code(db, code)
    listing['date'] = date
    listing['stalls'] = []
    for stall in fetch_stalls_by_location(db, code):
        listing['stalls'].append({
            "stallId": stall['stallId'],
            "name": stall['name'],
            "type": stall['type'],
            "image": stall['image'],
            "available": True,
            "food": [{**food, 'quantity': -1} for food in stall['food'] if food['available']]
        })
    listing['orderAvailable'] = orderAvailable
    listing['meal'] = meal
    return db.dailyListing.insert_one(listing).inserted_id

def del_listing(db, date):
    return db.dailyListing.delete_one({"date": date}).deleted_count

def update_stall_availability(db, date, stallId, availability):
    listing = fetch_listing(db, date)
    if not listing:
        raise Exception("Listing unavailable for this date")
    stalls = listing['stalls']
    for stall in stalls:
        if stall['stallId'] == stallId:
            stall['available'] = availability
            updated = db.dailyListing.update_one({"date": date}, {"$set": {"stalls": stalls}}).modified_count
            return updated
    raise Exception("No such stall in this listing")
    
def update_food_quantity(db, date, stallId, foodId, quantity):
    listing = fetch_listing(db, date)
    if not listing:
        raise Exception("Listing unavailable for this date")
    stalls = listing['stalls']
    for stall in stalls:
        if stall['stallId'] == stallId:
            for food in stall['food']:
                if food['id'] == foodId:
                    food['quantity'] = quantity
                    updated = db.dailyListing.update_one({"date": date}, {"$set": {"stalls": stalls}}).modified_count
                    return updated
    raise Exception("No such stall or food in this listing")
    
def fetch_stall_for_date(db, stallId, date, _id=0):
    stall = db.dailyListing.find_one({"date": date}, {"stalls": {"$elemMatch": {"stallId": stallId}}, "_id": _id})['stalls'][0]
    new_info = fetch_stall_by_id(db, stall['stallId'])
    stall['about'] = new_info['about']
    stall['contact'] = new_info['contact']
    stall['stallNo'] = new_info['stallNo']
    return stall

In [16]:
[doc for doc in db.stall.find({"location":"THM"}, {"_id":0, "stallId":1, "name":1, "stallNo": 1}, sort=[("stallId",1)])]

[{'name': 'Cafe 66', 'stallNo': '01-01', 'stallId': 'THM_01-01'},
 {'name': 'Mei Wei Xiao Chi', 'stallNo': '01-02', 'stallId': 'THM_01-02'},
 {'name': 'Da Xi Hainanese Chicken Rice Stall',
  'stallNo': '01-19',
  'stallId': 'THM_01-19'}]

In [145]:
update_stall_availability(db, "16052020", "THM_01-01", True)

1

In [68]:
date = "17052020"
stallId = "THM_01-01"
_id = 0
fetch_stall_for_date(db, stallId, date)

{'stallId': 'THM_01-01',
 'name': 'Cafe 66',
 'type': ['Drinks'],
 'image': '',
 'available': False,
 'food': [{'name': "Coffee 'O'",
   'id': 1,
   'description': 'Black coffee',
   'price': 0.7,
   'image': '',
   'available': True,
   'quantity': -1},
  {'name': 'Coffee w Milk',
   'id': 2,
   'description': '',
   'price': 0.9,
   'image': '',
   'available': True,
   'quantity': -1},
  {'name': "Tea 'O'",
   'id': 3,
   'description': 'Black tea',
   'price': 0.7,
   'image': '',
   'available': True,
   'quantity': -1},
  {'name': 'Tea w Milk',
   'id': 4,
   'description': '',
   'price': 0.9,
   'image': '',
   'available': True,
   'quantity': -1},
  {'name': "Milo 'O'",
   'id': 5,
   'description': '',
   'price': 1,
   'image': '',
   'available': True,
   'quantity': -1},
  {'name': 'Milo w Milk',
   'id': 6,
   'description': '',
   'price': 1.1,
   'image': '',
   'available': True,
   'quantity': -1},
  {'name': 'Iced Milo',
   'id': 7,
   'description': '',
   'price':

In [104]:
import datetime

def insert_user(db, aws_id, name, phone, email, payment_accounts=[]):
    if fetch_user(db, aws_id):
        raise Exception("User with this ID already exists!")
    user = {
        "awsId": aws_id,
        "name": name,
        "phone": phone,
        "email": email,
        "payment": payment_accounts,
        "dateJoined": datetime.datetime.now()
    }
    inserted_user = db.user.insert_one(user).inserted_id
    return inserted_user
    
def fetch_user(db, aws_id, _id=0):
    return db.user.find_one({"awsId": aws_id}, {"_id": _id})

def check_payment_method(db, method, username):
    return len([x for x in db.user.find({"payment.method": method, "payment.username": username}, {"_id":1}).limit(1)])==0

def insert_user_payment(db, aws_id, method, username):
    if len([x for x in db.user.find({"awsId": aws_id, "payment.method": method}, {"_id": 1}).limit(1)]) > 0:
        raise Exception("Payment method {} already exists for user".format(method))
    return db.user.update_one({"awsId": aws_id}, {"$push": {"payment": {
        "method": method,
        "username": username
    }}}).modified_count

def delete_user_payment(db, aws_id, method, username):
    if len([x for x in db.user.find({"awsId": aws_id, "payment.method": method, "payment.username": username}, {"_id": 1}).limit(1)])==0:
        raise Exception("Payment method {} with this username does not exist for user".format(method))
    return db.user.update_one({"awsId": aws_id}, {"$pull": {"payment": {"method": method, "username": username}}}).modified_count

def update_user_payment(db, aws_id, method, username):
    if len([x for x in db.user.find({"awsId": aws_id, "payment.method": method}, {"_id": 1}).limit(1)])==0:
        raise Exception("Payment method {} does not exist for user".format(method))
    return db.user.update_one({"awsId": aws_id, "payment.method": method}, {"$set": {"payment.$.username": username}}).modified_count

In [41]:
insert_user(db, "981b4aa4-9d51-44e9-806f-d4a520b468d5", "Gabriel Loye", "+91688706", "gabrielloye@hotmail.com", payment_accounts=[])

ObjectId('5ec2a3af303aa9d086371274')

In [47]:
[x for x in db.user.find({"awsId": "981b4aa4-9d51-44e9-806f-d4a520b468d5", "payment.method": "PayLah!"}, {"_id": 1}).limit(1)]

[]

In [90]:
x = insert_user_payment(db, "981b4aa4-9d51-44e9-806f-d4a520b468d5", "PayLah!", "abc")
x

1

In [100]:
fetch_user(db, "981b4aa4-9d51-44e9-806f-d4a520b468d5")

{'awsId': '981b4aa4-9d51-44e9-806f-d4a520b468d5',
 'name': 'Gabriel Loye',
 'phone': '+91688706',
 'email': 'gabrielloye@hotmail.com',
 'payment': [{'method': 'PayLah!', 'username': 'TEST'}],
 'dateJoined': datetime.datetime(2020, 5, 18, 23, 3, 11, 685000)}

In [99]:
aws_id = "981b4aa4-9d51-44e9-806f-d4a520b468d5"
method = "PayLah!"
update_user_payment(db, aws_id, method, "TEST")

1

In [103]:
check_payment_method(db, method, "TEST")

True

In [122]:
def fetch_transactions_by_date_meal(db, date, meal):
    transactions = []
    for doc in db.transaction.find({"date": date, "meal": meal}):
        doc['_id'] = str(doc['_id'])
        transactions.append(doc)
    return transactions

def insert_transaction(db, awsId, date, cart, paymentMethod, paymentUsername, meal, paid=False):
    if not fetch_user(db, aws_id):
        raise Exception("User with this ID already exists!")
    transaction = {
        "dateTime": datetime.datetime.now(),
        "date": date,
        "awsId": awsId,
        "cart": cart,
        "paid": paid,
        "paymentMethod": paymentMethod,
        "paymentUsername": paymentUsername,
        "totalPrice": sum([food['price'] for food in cart]),
        "meal": meal
    }
    inserted_id = db.transaction.insert_one(transaction).inserted_id
    return inserted_id

def update_transaction(db, _id, paid):
    return db.transaction.update_one({"_id": ObjectId(_id)}, {"$set": {"paid": paid}}).modified_count

def fetch_transactions_for_user(db, awsId):
    transactions = []
    for doc in db.transaction.find({"awsId": awsId}):
        doc['_id'] = str(doc['_id'])
        transactions.append(doc)
    return transactions

In [118]:
update_transaction(db, "5ec2b629303aa9d086371276", True)

1

In [123]:
fetch_transactions_by_date_meal(db, "15052020", "lunch")

[{'_id': '5ec2b629303aa9d086371276',
  'dateTime': datetime.datetime(2020, 5, 19, 0, 22, 1, 671000),
  'date': '15052020',
  'awsId': '981b4aa4-9d51-44e9-806f-d4a520b468d5',
  'cart': [],
  'paid': True,
  'paymentMethod': 'PayLah!',
  'paymentUsername': 'abc',
  'totalPrice': 0,
  'meal': 'lunch'}]

In [3]:
db.dailyListing.find_one({"date":"20052020"})

{'_id': ObjectId('5ec4cd09d1b73ab1102dd447'),
 'name': 'Tanglin Halt Market',
 'code': 'THM',
 'address': '48A Tanglin Halt Rd, Singapore 148813',
 'image': '',
 'date': '20052020',
 'stalls': [{'stallId': 'THM_01-01',
   'name': 'Cafe 66',
   'type': ['Drinks'],
   'image': '',
   'available': True,
   'food': [{'available': True,
     'description': 'Black coffee',
     'id': 1,
     'image': '',
     'name': "Coffee 'O'",
     'price': 0.7,
     'margin': 0.2,
     'quantity': -1},
    {'available': True,
     'description': '',
     'id': 2,
     'image': '',
     'name': 'Coffee w Milk',
     'price': 0.9,
     'margin': 0.2,
     'quantity': -1},
    {'available': True,
     'description': 'Black tea',
     'id': 3,
     'image': '',
     'name': "Tea 'O'",
     'price': 0.7,
     'margin': 0.2,
     'quantity': -1},
    {'available': True,
     'description': '',
     'id': 4,
     'image': '',
     'name': 'Tea w Milk',
     'price': 0.9,
     'margin': 0.2,
     'quantity': -1

In [39]:
db.dailyListing.find_one({"date": "20052020"})

{'_id': ObjectId('5ec41d010aed8901da2f83ea'),
 'name': 'Tanglin Halt Market',
 'code': 'THM',
 'address': '48A Tanglin Halt Rd, Singapore 148813',
 'image': '',
 'date': '20052020',
 'stalls': [{'stallId': 'THM_01-01',
   'name': 'Cafe 66',
   'type': ['Drinks'],
   'image': '',
   'available': True,
   'food': [{'available': True,
     'description': 'Black coffee',
     'id': 1,
     'image': '',
     'name': "Coffee 'O'",
     'price': 0.7,
     'margin': 0.2,
     'quantity': -1},
    {'available': True,
     'description': '',
     'id': 2,
     'image': '',
     'name': 'Coffee w Milk',
     'price': 0.9,
     'margin': 0.2,
     'quantity': -1},
    {'available': True,
     'description': 'Black tea',
     'id': 3,
     'image': '',
     'name': "Tea 'O'",
     'price': 0.7,
     'margin': 0.2,
     'quantity': -1},
    {'available': True,
     'description': '',
     'id': 4,
     'image': '',
     'name': 'Tea w Milk',
     'price': 0.9,
     'margin': 0.2,
     'quantity': -1

In [7]:
[x for x in db.user.find({})]

[{'_id': ObjectId('5ec2c880157cf101e9e9be45'),
  'awsId': 'c21c9516-cfbf-4ff7-8a3b-641701c2fb72',
  'name': 'Gabriel Loye',
  'phone': '+6591688706',
  'email': 'gabrielloye@hotmail.com',
  'payment': [],
  'dateJoined': datetime.datetime(2020, 5, 18, 17, 40, 16, 315000)},
 {'_id': ObjectId('5ec364c90d9cda0f35c8dbcd'),
  'awsId': '7889de8a-0028-47bb-a26e-d30f9a05c5d6',
  'name': 'Jeremy Loye',
  'phone': '+6597800777',
  'email': 'jeremyloye@gmail.com',
  'payment': [{'method': 'PayLah!', 'username': 'Jeremy Loye'}],
  'dateJoined': datetime.datetime(2020, 5, 19, 4, 47, 5, 850000)},
 {'_id': ObjectId('5ec407551add2b43fe93596f'),
  'awsId': '9151cd2c-78a4-4867-9d09-3b7e86d4f7a2',
  'name': 'Donovan Liew',
  'phone': '+6597974685',
  'email': 'donovanlyh@gmail.com',
  'payment': [{'method': 'PayLah!', 'username': 'PL4685'},
   {'method': 'PayNow', 'username': 'PN4685'}],
  'dateJoined': datetime.datetime(2020, 5, 19, 16, 20, 37, 14000)},
 {'_id': ObjectId('5ec53ce4a6d00525292bd07c'),
  '

In [6]:
import requests
requests.post("https://hb65mr6g85.execute-api.ap-southeast-1.amazonaws.com/dev/users/add", json={
    "awsId": "1bcde240-99c4-4366-a03f-02879a98a499",
    "name": "Arnold Teo",
    "phone": "+6596927057",
    "email": "arnoldkunho@hotmail.com",
    "payment": []
})

<Response [200]>